In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"madhandev","key":"1c65c54080ec9ddaaf7af8ec6d98ae74"}'}

In [ ]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Dec  5 12:28 kaggle.json


In [ ]:
!pip install -q kaggle

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
cd /content/drive/MyDrive/Humans

/content/drive/MyDrive/Humans


In [ ]:
pwd

'/content/drive/MyDrive/Humans'

In [ ]:
!kaggle datasets list


In [ ]:
!kaggle datasets download -d ashwingupta3012/human-faces

In [ ]:
!unzip human-faces.zip

In [ ]:
!pip install tensorflow-gpu==2.8.0

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.image import array_to_img

In [ ]:
gpus =tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
data_dir = '/content/drive/MyDrive/Humans'
img_list = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
for i in os.listdir(data_dir):
    print(i)

sample = ""

In [ ]:
for i in os.listdir(data_dir):
  image_path=os.path.join(data_dir,i);
  try:
    img=tf.keras.preprocessing.image.load_img(image_path)
    img=tf.keras.preprocessing.image.img_to_array(img)
    tip=img.dtype
    if tip not in img_list:
      print("Image type not in image list")
      print(i)
      os.remove(image_path)
  except Exception as e:
    print("Issue {}".format(e))

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Humans',
    image_size=(256, 256),
    shuffle=True,
)

Found 7219 files belonging to 1 classes.


In [ ]:
data = data.map(lambda x, y: (x / 255, y))

In [ ]:
data

<MapDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
scaled_itt=data.as_numpy_iterator()
batch=scaled_itt.next()

In [ ]:
fig, ax =plt.subplots(ncols=4,figsize=(28,28))
for idx,img in enumerate(batch[0][:4]):
  ax[idx].imshow(img)
  ax[idx].title.set_text(batch[1][idx])

In [ ]:
def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(256, input_dim=100, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(256 * 256 * 3, activation='sigmoid'))
    model.add(layers.Reshape((256, 256, 3)))
    return model

In [ ]:
generator = build_generator()
random_noise = tf.random.normal((4, 100))
generated_images = generator(random_noise)

In [ ]:
generatore_images=generator(tf.random.normal((4,100)))
generatore_images.shape

TensorShape([4, 256, 256, 3])

In [ ]:
for i in range(5):
  fake_img = generator(tf.random.normal((4, 100)), training=False)
  print(fake_img)
  print(i, "end")

In [ ]:
fake_img.shape

TensorShape([32, 256, 256, 3])

In [ ]:
generator.summary()

In [ ]:
def build_discriminator(in_shape):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=in_shape))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.4))  # Add dropout for regularization
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
in_shape = (256, 256, 3)
discriminator = build_discriminator(in_shape)
discriminator.summary()
discriminator.predict(np.expand_dims(generated_images[0], 0))

In [ ]:
image_from_generator = generator(tf.random.normal((32,100)), training=True)
len(image_from_generator)

32

In [ ]:
count=0
while(count==0):
  image_from_generator = generator(tf.random.normal((32,100)), training=True)
  print(image_from_generator)
  print(image_from_generator.shape)

  prediction=discriminator(image_from_generator, training=True)
  print("label")
  print(prediction)

  image_from_generator *= 255
  # image_from_generator = image_from_generator.astype('uint8')
  for i in range(32):
    img = array_to_img(image_from_generator[i])
    img.save(os.path.join('/content/drive/MyDrive/GAN_generated_images', f'generated_img_{i}.png'))

  for j in prediction:
    if int(j[0])==0:
      count=1

In [ ]:
imge=generator.predict(tf.random.normal((32,100)))
imge.shape

In [ ]:
predicted_label=discriminator(imge, training=True)
len(predicted_label)

32

In [ ]:
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                      loss='binary_crossentropy',  # Use binary cross-entropy for GANs
                      metrics=['accuracy'])

In [ ]:
class ModelCallback(Callback):
    def __init__(self, num_img=3, latent_dim=100):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        gen_imgs = generator.predict(tf.random.normal((self.num_img, self.latent_dim)))
        gen_imgs *= 255
        gen_imgs = gen_imgs.astype('uint8')
        for i in range(self.num_img):
            img = array_to_img(gen_imgs[i])
            img.save(os.path.join('/content/drive/MyDrive/gen_img', f'generated_img_{epoch}_{i}.png'))

In [ ]:
class GAN(models.Model):
    def __init__(self, discriminator, generator, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.discriminator = discriminator
        self.generator = generator

    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs):
        super().compile(*args, **kwargs)
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss

    def train_step(self, batch):
        real_img, real_img_label = batch
        fake_img = self.generator(tf.random.normal((32, 100)), training=False)

        # Train discriminator
        with tf.GradientTape() as disc_tape:
            what_real = self.discriminator(real_img, training=True)
            what_fake = self.discriminator(fake_img, training=True)

            what_real_fake = tf.concat([what_real, what_fake], axis=0)

            # Create label
            y_real_fake = tf.ones_like(what_real_fake)

            # Noise
            noise_real = 0.15 * tf.random.uniform(tf.shape(what_real))
            noise_fake = -0.15 * tf.random.uniform(tf.shape(what_fake))

            what_real_fake_noise_cal = tf.concat([what_real+noise_real, what_fake+noise_fake], axis=0)

            # Calculate losses
            total_d_loss = self.d_loss(y_real_fake, what_real_fake_noise_cal)

        gradients_of_discriminator = disc_tape.gradient(total_d_loss, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        # Train generator
        with tf.GradientTape() as gen_tape:
            gen_img = self.generator(tf.random.normal((32, 100)), training=True)
            # Create the predicted label
            predicted_label = self.discriminator(gen_img, training=True)
            total_g_loss = self.g_loss(tf.ones_like(predicted_label), predicted_label)

        # Apply backpropagation
        gradients_of_generator = gen_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))

        return {
            'd_loss': total_d_loss,
            'g_loss': total_g_loss
        }


In [ ]:
img_gan = GAN(discriminator, generator)

In [ ]:
g_opt = optimizers.Adam(learning_rate=0.0001)
d_opt = optimizers.Adam(learning_rate=0.00001)
g_loss = tf.keras.losses.BinaryCrossentropy()
d_loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
img_gan.compile(g_opt, d_opt, g_loss, d_loss)

In [ ]:
callback = ModelCallback()

In [ ]:
hist = img_gan.fit(data, epochs=20, callbacks=[callback], batch_size=8)

In [ ]:
imgs = generator.predict(tf.random.normal((32, 100)))

In [ ]:
plt.suptitle('Loss')
plt.plot(hist.history['d_loss'], label='d_loss')
plt.plot(hist.history['g_loss'], label='g_loss')
plt.legend()
plt.show()